In [1]:
import pandas as pd
from lxml import etree
import requests

---- using HTML response ----

In [2]:
url = "https://www.hemnet.se/bostader"
params= {
    'housing_form_groups':'apartments',
    'location_ids':'18031',
    'item_types':'bostadsratt'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.url

'https://www.hemnet.se/bostader?housing_form_groups=apartments&location_ids=18031&item_types=bostadsratt'

--------- w beautiful soup ------------

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

--- by using the jsons in the script tags we can get locations and listings ----

In [5]:
import json

#find all script tags with json data
script_list=soup.find_all("script",type="application/ld+json")

#json.loads(script_list[1].text)
#type(script_list)

In [6]:
#get the two jsons we want - place and item list
place_json=json.loads(script_list[1].text)
item_list_json=json.loads(script_list[2].text)

In [10]:
#put item list json into a df
item_list_df=pd.json_normalize(item_list_json['itemListElement'])
item_list_df

,@type,url,position
0,ListItem,https://www.hemnet.se/bostad/radhus-5rum-becko...,1
1,ListItem,https://www.hemnet.se/bostad/lagenhet-5rum-ost...,2
2,ListItem,https://www.hemnet.se/bostad/lagenhet-4rum-sod...,3
3,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-sto...,4
4,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-kis...,5
5,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-sto...,6
6,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-sto...,7
7,ListItem,https://www.hemnet.se/bostad/lagenhet-2rum-asp...,8
8,ListItem,"https://www.hemnet.se/bostad/lagenhet-1,5rum-l...",9
9,ListItem,https://www.hemnet.se/bostad/ovrigt-1rum-soder...,10


In [8]:
#and location json into another df
place_df=pd.json_normalize(place_json)
place_df

,@context,@type,address.addressLocality,address.addressRegion,address.addressCountry
0,http://schema.org,Place,Stockholms kommun,Stockholms län,SE


In [9]:
#make new col with location/region for the list items
new_cols={'addressLocality':place_df.iloc[0]['address.addressLocality'],'addressRegion':place_df.iloc[0]['address.addressRegion']}
item_list_df.assign(**new_cols)

,@type,url,position,addressLocality,addressRegion
0,ListItem,https://www.hemnet.se/bostad/radhus-5rum-becko...,1,Stockholms kommun,Stockholms län
1,ListItem,https://www.hemnet.se/bostad/lagenhet-5rum-ost...,2,Stockholms kommun,Stockholms län
2,ListItem,https://www.hemnet.se/bostad/lagenhet-4rum-sod...,3,Stockholms kommun,Stockholms län
3,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-sto...,4,Stockholms kommun,Stockholms län
4,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-kis...,5,Stockholms kommun,Stockholms län
5,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-sto...,6,Stockholms kommun,Stockholms län
6,ListItem,https://www.hemnet.se/bostad/lagenhet-3rum-sto...,7,Stockholms kommun,Stockholms län
7,ListItem,https://www.hemnet.se/bostad/lagenhet-2rum-asp...,8,Stockholms kommun,Stockholms län
8,ListItem,"https://www.hemnet.se/bostad/lagenhet-1,5rum-l...",9,Stockholms kommun,Stockholms län
9,ListItem,https://www.hemnet.se/bostad/ovrigt-1rum-soder...,10,Stockholms kommun,Stockholms län
